# Training Football Opjects Detector

## Configuration

<span style="color: yellow;">To run training of our models we need to copy this file into google colab and run it there.</span> If you want to use it in other environment, you need to change the code.


### Install dependencies

In [1]:
!pip install -q ultralytics==8.0.196
!pip install -q roboflow
!pip install -q torch

### Check GPU access


In [ ]:
!nvidia-smi

### Imports

In [46]:
from roboflow import Roboflow
import shutil
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



In [ ]:
import os
HOME = os.getcwd()
print(HOME)

### Colab imports
We should use colab to train our model. However if you want to test the code locally it is possible but some functions will not be available and training will be slower.

In [5]:
try:
    from google.colab import drive
    from google.colab import userdata
    in_colab = True
except ImportError:
    from dotenv import load_dotenv
    in_colab = False

# Load dataset


In [6]:
dataset_version = 1
dataset_location = "../data/training"

def load_dataset(model_name):
    if in_colab:
        rf_api_key = userdata.get("ROBOFLOW_API_KEY")
    else:
        load_dotenv(dotenv_path='../config/.env')
        rf_api_key = os.getenv("ROBOFLOW_API_KEY")

    rf = Roboflow(api_key=rf_api_key)
    project = rf.workspace("k-rzeznicki").project("football-players-detection-3zvbc-fynld")
    version = project.version(dataset_version)
    
    version.download(model_name, location=dataset_location)
    print("loaded dataset to", dataset_location)

## Deploy weights
Deploy weights of the trained model to roboflow

In [7]:
def deploy_weights(model_name, results_folder, weights_filename = "best.pt"):
    if in_colab:
        rf_api_key = userdata.get("ROBOFLOW_API_KEY")
    else:
        load_dotenv(dotenv_path='../config/.env')
        rf_api_key = os.getenv("ROBOFLOW_API_KEY")

    rf = Roboflow(api_key=rf_api_key)
    project = rf.workspace("k-rzeznicki").project("football-players-detection-3zvbc-fynld")
    version = project.version(dataset_version)
    version.deploy(model_name, results_folder, weights_filename)


# Training

First we will create a functions that will help us to train our models

###  Function to train yolo model

In [8]:

def train_yolo(model_name, epochs, imgsz, train_folder = None, gpu = True):
    """
    This function trains a YOLO model using the specified parameters.

    Parameters:
    model_name (str): The name of the model to be trained.
    epochs (int): The number of training epochs.
    imgsz (int or tuple): The size of the images for training.
    train_folder (str, optional): The folder where the training results will be saved. Defaults to None.
    gpu (bool, optional): Flag to indicate whether to use GPU for training. Defaults to True.

    Raises:
    SystemError: If GPU is not available and gpu is set to True.
    """
    # Check if GPU is available
    if torch.cuda.is_available():
        device = 0
    elif torch.backends.mps.is_available() and not gpu:
        device = "mps"
    elif gpu:
        raise SystemError('GPU device not found')
    else:
        device = "cpu"
        
    yolo = YOLO(model_name)
    yolo.train(data=f'{dataset_location}/data.yaml', epochs=epochs, imgsz=imgsz, project=train_folder, name=model_name, exist_ok=True, batch=6, device=device)
    



In [9]:
def validate_model(model_name, imgsz, train_folder = None, iou = 0.6, conf = 0.001, gpu = True):
    """
    This function validates trained YOLO model based on our validation dataset using the specified parameters and metrics.

    Parameters:
    model_name (str): The name of the model to be validated.
    imgsz (int or tuple): The size of the images for validation.
    train_folder (str, optional): The folder where our dataset and .yaml file is stored. Defaults to None.
    iou (float, optional): The Intersection over Union threshold for validation. Defaults to 0.6.
    conf (float, optional): The confidence threshold for validation. Defaults to 0.001.
    gpu (bool, optional): Flag to indicate whether to use GPU for validation. Defaults to True.

    Raises:
    SystemError: If GPU is not available and gpu is set to True.

    Returns:
    dict: A dictionary containing validation metrics such as mAP (mean Average Precision).
    """
    if torch.cuda.is_available():
        device = 0
    elif torch.backends.mps.is_available() and not gpu:
        device = "mps"
    elif gpu:
        raise SystemError('GPU device not found')
    else:
        device = "cpu"
    
    model = YOLO(model_name)  # load an official model
    model = YOLO(train_folder + "/weights/best.pt")  # load a custom model

    destination_folder = dataset_location + "/validation"
    # Validate the model
    metrics = model.val(imgsz=imgsz, data=f'{dataset_location}/data.yaml', iou=iou, conf=conf, save_json=True, plots=True, project=destination_folder, device=device)  # no arguments needed, dataset and settings remembered
    return metrics

### copy trained model weights and all run files to google drive
This function will copy all the files from the run folder to the google drive.

In [10]:
def copy_to_drive(source_folder, destination_folder):
    if not in_colab:
        raise SystemError('You can copy files to drive only in Google Colab')
    drive.mount('/content/drive')

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Copy the entire folder recursively
    try:
        shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
        print(f"Successfully copied '{source_folder}' to '{destination_folder}'")
    except Exception as e:
        print(f"Error copying folder: {e}")
    drive.flush_and_unmount()

## Run training

In [ ]:
# choose model and parameters
model_name = "yolov8" # yolo11
roboflow_name = "yolov8" #yolov11 # ussually the same as model name but for yolov11 the name convantion changed 
model_size = "n" # x
epochs = 5 # 100
imgsz = 180 # 1280
project_name = "football_analysis"

model = model_name + model_size
train_folder = "../runs/train/" + project_name
destination_folder = train_folder

# load dataset
load_dataset(roboflow_name)

# train model
train_yolo(model_name=model, epochs=epochs, imgsz=imgsz, train_folder=train_folder, gpu=False)

destination_folder = destination_folder + "/" + model
weights_folder = destination_folder +  "/weights"

# deploy weights
deploy_weights(model_name=roboflow_name, results_folder=weights_folder, weights_filename="best.pt")

# copy results to gooogle drive
if in_colab:
    drive_folder = "/content/drive/MyDrive/Colab/" + project_name + "/" + model
    copy_to_drive(source_folder=train_folder, destination_folder=drive_folder)



## Validate model

In [ ]:
validate_model(model_name=model, imgsz=imgsz, train_folder=destination_folder, gpu=False)

## Show results

In [ ]:
img_path = f'{destination_folder}/confusion_matrix.png'
print(img_path)
if os.path.exists(img_path):
    img = Image.open(img_path)
    image = mpimg.imread(img_path)
    plt.imshow(image)
    plt.show()
else:
    print(f"Image not found at {img_path}")

In [ ]:
img_path = f'{destination_folder}/results.png'
if os.path.exists(img_path):
    img = Image.open(img_path)
    plt.imshow(img)
    plt.show()
else:
    print(f"Image not found at {img_path}")